In [6]:
import meshplot as mp
import numpy as np

In [28]:
def hexs2tets(hexs, diagmode=0):
    tets = np.empty((len(hexs)*5,4),dtype=np.int)
    T2H = np.empty((len(hexs)*5,),dtype=np.int)
    for i in range(len(hexs)):
        if diagmode == 0:
            diagtype = np.random.randint(2)
        elif diagmode == 1:
            diagtype = 0
        else:
            diagtype = i % 2 == 0
        
        if diagtype == 0:
            tets[5*i + 0] = [hexs[i][j] for j in [0,1,2,5]]
            tets[5*i + 1] = [hexs[i][j] for j in [0,2,3,7]]
            tets[5*i + 2] = [hexs[i][j] for j in [2,6,7,5]]
            tets[5*i + 3] = [hexs[i][j] for j in [0,5,7,4]]
            tets[5*i + 4] = [hexs[i][j] for j in [0,2,7,5]]
        else:
            tets[5*i + 0] = [hexs[i][j] for j in [0,1,3,4]]
            tets[5*i + 1] = [hexs[i][j] for j in [1,2,3,6]]
            tets[5*i + 2] = [hexs[i][j] for j in [3,6,7,4]]
            tets[5*i + 3] = [hexs[i][j] for j in [1,6,4,5]]
            tets[5*i + 4] = [hexs[i][j] for j in [1,6,3,4]]
        T2H[5*i:5*(i+1)] = i
    return tets, T2H
        

def tets2tris(tets):
    tris = np.empty((len(tets)*4,3),dtype=np.int)
    for i in range(len(tets)):
        tris[4*i + 0] = [tets[i][j] for j in [0,1,2]]
        tris[4*i + 1] = [tets[i][j] for j in [0,1,3]]
        tris[4*i + 2] = [tets[i][j] for j in [1,2,3]]
        tris[4*i + 3] = [tets[i][j] for j in [2,0,3]]
    return tris

def generate_hex_grid(DX,N,center=True):
    n,m,l = N
    di,dj,dk = DX
    V = np.empty(((n+1)*(m+1)*(l+1), 3), dtype=np.float)
    
#     X,Y,Z = np.meshgrid(np.linspace(0,di*n,n+1),np.linspace(0,dj*m,m+1),np.linspace(0,dk*l,l+1))
#     V[:,0] = X.reshape(V.shape[0])
#     V[:,1] = Y.reshape(V.shape[0])
#     V[:,2] = Z.reshape(V.shape[0])
        
    for i in range(n+1):
        I = i*(m+1)*(l+1)
        for j in range(m+1):
            J = j*(l+1)
            for k in range(l+1):
                V[I+J+k] = [i*di,j*dj,k*dk]
        
    if center:
        V[:] -= [di*n*0.5,dj*m*0.5,dk*l*0.5]
        
    def vix(i,j,k):
        return i*(m+1)*(l+1) + j*(l+1) + k
        
    H = np.empty((n*m*l,8),dtype=np.int)
    for i in range(n):
        I = i*m*l
        for j in range(m):
            J = j*l
            for k in range(l):
                H[I+J+k] = [vix(i+0,j+0,k+0),
                            vix(i+1,j+0,k+0),
                            vix(i+1,j+0,k+1),
                            vix(i+0,j+0,k+1),
                            vix(i+0,j+1,k+0),
                            vix(i+1,j+1,k+0),
                            vix(i+1,j+1,k+1),
                            vix(i+0,j+1,k+1)]
    return V, H

# note default set existing: allElements
def vega_file_string(V,T,M,S):
    assert(len(S)==len(M))
    nverts = len(V)
    ntets = len(T)
    setname = lambda i: "set_" + str(i)
    matname = lambda i: "mat_" + str(i)
    
    # # Vega mesh file.
    # # NUMVERT vertices, NUMTET elements
    s = "# Vega mesh file\n# %d vertices, %d elements\n" % (nverts, ntets)
    s += "\n"
    
    # *VERTICES
    # NUMVERTS 3 0 0
    # [VIX X Y Z # VIX STARTING AT 0(or 1)]_rows
    s += "*VERTICES\n%d 3 0 0\n" % nverts
    for i,v in enumerate(V):
        s += "%d %f %f %f\n" % (i, v[0], v[1], v[2])
    s += "\n"
    
    # *ELEMENTS
    # TET
    # NUMTETS 4 0
    # [TIX VIX0 VIX1 VIX2 VIX3]_rows
    s += "*ELEMENTS\nTET\n%d 4 0\n" % ntets
    for i,t in enumerate(T):
        s += "%d %d %d %d %d\n" % (i, t[0], t[1], t[2], t[3])
    s += "\n"
        
    # # MATERIALNAMEi material
    # *MATERIAL MATERIALNAMEi
    # ENU, 1000, 25000000, 0.45 # density (kg/m^3), Young's modulus (N/m^2), Poisson's ratio
    for i,m in enumerate(M):
        s += "*MATERIAL %s\n" % matname(i)
        s += "ENU, %g, %g, %g\n\n" % (m[0],m[1],m[2])

    # # SETNAMEi elements
    # *SET SETNAMEi
    # TIX0, TIX1, TIX2 # sorted commaseparated indices (linebreaks ignored)
    # TIX3, TIX4, ...
    for i,st in enumerate(S):
        name = setname(i)
        s += "# %s elements\n*SET %s\n" % (name, name)
        rows = []
        for row in range(len(st)//8):
            rows.append(", ".join([str(st[row*8+j]) for j in range(8)]))
        nrest = len(st) % 8
        if nrest > 0:
            rows.append(", ".join([str(st[len(st)-nrest+j]) for j in range(nrest)]))
        s += ",\n".join(rows)
        s += "\n\n"

    # # link SETNAMEi to MATERIALNAMEi
    # *REGION
    # SETNAMEi, MATERIALNAMEi
    for i in range(len(M)):
        s += "# link %s to %s\n" % (setname(i), matname(i))
        s += "*REGION\n%s, %s\n\n" % (setname(i), matname(i))
    return s

def get_boundary_verts():
    pass

def filter_boundary_faces(F, boundary_verts):
    pass

In [29]:
np.random.seed(1991)
V, H = generate_hex_grid(DX=(0.1,0.1,0.1),N=(6,2,4),center=True)
material_types = [0, 1]
M = np.random.choice(material_types,size=(6,2,4)).reshape(-1)
T, T2H = hexs2tets(H, diagmode=0)
M = M[T2H] # get tet material ids from associated hex ids

# collect set of tets with same material
S = []
for i in range(len(material_types)):
    S.append(sorted(np.where(M == material_types[i])[0]))
    
# define materials
M = [[1000, 1e6, 0.4], [2000, 5e6, 0.4]]

# F = tets2tris(T)
# mp.plot(V, F, shading={"wireframe": True})

In [30]:
filestr = vega_file_string(V,T,M,S)
with open("../meshes/testmesh.veg",'w') as file:
    file.write(filestr)

In [ ]:

V = np.random.uniform(0,1,(10,3))
T = np.array([[0,1,2,3],[3,4,5,1]])
S = np.array([[0,1,8],[1,2,3,4,5,6,7,8,9,10,11,12,14,15,1,5,3,2,6,8,6,4,4,3,2,4,]])
M = np.array([[1000, 1e6, 0.4],[500, 2e6, 0.4]])

print(vega_file_string(V,T,M,S))
# py define material
# py define voxel material ID

In [ ]:
V = np.array([
    [0,0,0],
    [0,0,1],
    [0,1,1],
    [0,1,0],
    [1,0,0],
    [1,0,1],
    [1,1,1],
    [1,1,0],
], dtype = np.float)
H = np.array([[0,1,2,3,4,5,6,7]],dtype=np.int)
T,T2H = hexs2tets(H, diagmode=0)
F = tets2tris(T)
F = tets2tris(T)
mp.plot(V, F, shading={"wireframe": True})

In [ ]:
V = np.random.uniform(-1,1,(4,3))*1
T = np.array([[0,1,2,3]],dtype=np.int)
F = tets2tris(T)
mp.plot(V, F, shading={"wireframe": True})